IMPORTANT:
SA - sent through email
Window - sent through email
3PD - sent through email
(Hours of Op - Fresh Insights - Gold Workbooks - Daily Open and Close Times)
EzCatering - sent through email
(OSAT - Sub Listens ranker - click three dots, Export)

Dependencies

In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from functools import reduce
import glob

Read in raw data files

In [3]:
# Read in all data
# IMPORTANT: switch file names to match script
win_data = pd.read_excel("data/win_data.xlsx")
ez_data = pd.read_excel("data/ez_data.xlsx")
osat_data = pd.read_excel("data/osat_data.xlsx")
sa_data = pd.read_excel("data/sa_data.xlsx")
hours_data = pd.read_csv("data/hours_data.csv")
threepd_data = pd.read_excel("data/3pd_data.xlsx")
bc_data = pd.read_excel("data/BC_list.xlsx")
rev_data = pd.read_csv("data/rev_data.csv")
drinks_data = pd.read_csv("data/drinks_data.csv")
rest_data = pd.read_csv("data/rest_data.csv")
ro_data = pd.read_csv("data/ro_data.csv")

Clean and Transform bc_data

In [56]:
# Concate BC List sheets into one
bc_data = pd.concat(pd.read_excel("data/BC_list.xlsx", sheet_name=None), 
                   ignore_index=True)

In [57]:
bc_data = bc_data[["Restaurant #", "Field Consultant"]]

In [58]:
bc_data = bc_data.rename(columns={"Restaurant #": "Store Number"})

In [59]:
bc_data["Field Consultant"].isnull().sum()

2

In [2]:
# Removed for privacy
# bc_data.drop_duplicates(subset=["Store Number"], keep="first")

In [61]:
bc_data.to_excel("data/BC_list.xlsx", index=False)

Clean and Transform hours_data

IMPORTANT: Delete first four rows from raw file

In [29]:
# h_data = pd.read_excel("data/hours_data_clean.xlsx")
# hours_data = pd.DataFrame(h_data)

In [8]:
# Create DF to rename columns
hours_data = pd.DataFrame(hours_data)
hours_data = hours_data.rename(columns={"Hour Type ": "Hour_Type"})
hours_data = hours_data.rename(columns={"Restaurant Num": "Store Number"})
hours_data = hours_data.rename(columns={"Total_Hours": "Total Hours"})

In [9]:
# Hours Data - Remove rows with Required Hours (must change col to Hour_Type)
hours_data = hours_data[hours_data.Hour_Type != "Required Hours"]

In [10]:
hours_data = hours_data[["Store Number", "Total Hours", "BD Contract", "Primary Owner Name", "Restaurant"]]

In [3]:
# Removed for privacy
# hours_data["Primary Owner Name"] = hours_data["Primary Owner Name"].replace("###", "###")

In [4]:
# Removed for privacy
# hours_data.drop_duplicates(subset=["Store Number"], keep="first")

In [19]:
# Write Clean set
hours_data.to_excel("data/hours_data_clean.xlsx", index=False)

Clean and Transform sa_data

In [11]:
# DF and Rename Columns
sa_data = pd.DataFrame(sa_data)
sa_data = sa_data.rename(columns={"Restaurant Number": "Store Number"})
sa_data = sa_data.rename(columns={"SA 1 # Overdue": "SA1 Overdue"})
sa_data = sa_data.rename(columns={"SA 1 % Complete": "SA1 Complete"})

In [12]:
sa_data = sa_data[["Store Number", "SA1 Overdue", "SA1 Complete"]]

In [13]:
# Drop bad duplicated data in sa_data
sa_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,SA1 Overdue,SA1 Complete
0,2,0,1.0
1,8,0,1.0
2,10,0,1.0
3,12,0,1.0
4,24,0,1.0
...,...,...,...
36528,71395,2,0.0
36529,71626,0,0.0
36530,71656,0,0.0
36531,71699,0,0.0


In [14]:
sa_data.to_excel("data/sa_data_clean.xlsx", index=False)

In [ ]:
# Unnamed Column
# sa_data.drop(["Unnamed: 0"], axis=1, inplace=True)

Clean and Transform osat_data

In [15]:
# DF and Rename Columns
osat_data = pd.DataFrame(osat_data)
osat_data = osat_data.rename(columns={"": "Store Number"})
osat_data = osat_data.rename(columns={"Taste/Quality of the Meal": "Quality"})
osat_data = osat_data.rename(columns={"Cleanliness of Restaurant": "Cleanliness"})
osat_data = osat_data.rename(columns={"Net Promoter Score": "NPS"})
osat_data = osat_data.rename(columns={"% Alerts Closed within 24 Hours": "Alerts Closed 24 Hours"})
osat_data = osat_data.rename(columns={"Surveys per Restaurant": "Surveys Per Restaurant"})

In [16]:
osat_data = osat_data.rename(columns={"Unnamed: 0": "Store Number"})

In [17]:
# Clean OSAT column 
osat_data['Store Number'] = osat_data['Store Number'].str.replace(r'\D', '')
osat_data["Store Number"] = osat_data["Store Number"].str[:-1]

C:\Users\Westjohn\AppData\Local\Temp/ipykernel_9064/415350303.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  osat_data['Store Number'] = osat_data['Store Number'].str.replace(r'\D', '')


In [18]:
# Drop Multiple Columns from DF
osat_data = osat_data[["Store Number", "Overall Satisfaction", "Quality", "Speed of Service", "Experience with Staff", "Cleanliness", "NPS", "Alerts Closed 24 Hours", "Surveys Per Restaurant"]]

In [19]:
# Drop bad duplicated data in sa_data
osat_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,Overall Satisfaction,Quality,Speed of Service,Experience with Staff,Cleanliness,NPS,Alerts Closed 24 Hours,Surveys Per Restaurant
0,10021,68.421053,68.421053,84.210526,63.157895,68.421053,52.631579,100.0,19.0
1,10029,70.0,75.0,90.0,80.0,75.0,65.0,100.0,20.0
2,10030,88.0,76.0,84.0,88.0,80.0,80.0,100.0,25.0
3,10070,64.285714,71.428571,78.571429,78.571429,64.285714,42.857143,100.0,14.0
4,10130,28.571429,42.857143,57.142857,42.857143,57.142857,0.0,40.0,7.0
...,...,...,...,...,...,...,...,...,...
627,7684,28.571429,57.142857,57.142857,57.142857,42.857143,0.0,100.0,7.0
628,7810,71.428571,78.571429,64.285714,50.0,85.714286,71.428571,100.0,14.0
629,7811,80.0,80.0,80.0,80.0,80.0,70.0,100.0,10.0
630,7842,58.823529,58.823529,82.352941,64.705882,52.941176,29.411765,100.0,17.0


In [20]:
osat_data.to_excel("data/osat_data_clean.xlsx", index=False)

Clean and transform threepd_data

In [18]:
# DF and Rename Columns
threepd_data = pd.DataFrame(threepd_data)
threepd_data = threepd_data.rename(columns={"DAI Store Number - ": "Store Number"})
threepd_data = threepd_data.rename(columns={"Grubhub Status?": "Grubhub Status"})
threepd_data = threepd_data.rename(columns={"Subway Delivery Live?": "Subway Delivery Status"})
threepd_data = threepd_data.rename(columns={"Doordash Status?": "Doordash Status"})

In [22]:
# Drop Columns
threepd_data = threepd_data[["Store Number", "Grubhub Status", "Subway Delivery Status", "Doordash Status", "Uber Status"]]

In [23]:
# Drop bad duplicated data in sa_data
threepd_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,Grubhub Status,Subway Delivery Status,Doordash Status,Uber Status
0,10070,Eligible,Yes,Live,Live
1,10130,In Process,Yes,Live,Live
2,10380,Eligible,Yes,Live,Live
3,10402,Not Eligible,NaN,Eligible,Eligible
4,10406,Live,Yes,Live,Live
...,...,...,...,...,...
633,11619,Not Eligible,Yes,Live,Not Eligible
634,21852,Live,Yes,Live,Live
635,45450,Not Eligible,NaN,Not Eligible,Eligible
636,60545,Live,Yes,Live,Not Eligible


In [24]:
threepd_data.to_excel("data/3pd_data_clean.xlsx", index=False)

Clean and transform win_data

IMPORTANT: Drop first 4 rows before moving forward

In [14]:
# DF and Rename Columns
win_data = pd.DataFrame(win_data)
win_data = win_data.rename(columns={"Restaurant Number": "Store Number"})
win_data = win_data.rename(columns={" % Complete": "Window Complete"})

In [15]:
# Drop Columns
win_data = win_data[["Store Number", "Window Complete"]]

In [16]:
# Drop bad duplicated data in sa_data
win_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,Window Complete
0,1,0.000000
1,2,1.000000
2,8,0.000000
3,10,0.600000
4,12,0.769231
...,...,...
21061,71231,0.545455
21062,71303,0.166667
21063,71385,0.264706
21064,71392,0.050000


In [17]:
win_data.to_excel("data/win_data_clean.xlsx", index=False)

Clean and Transform ez_data

Use LocationsEnrolledwithezCater sheet

In [ ]:
ez_data = ez_data[["Store Number", "ezCater", "ezDelivery", "ezDispatch"]]

In [5]:
# Drop bad duplicated data in sa_data
# ez.drop_duplicates(subset=["Store Number"], keep="first")

Clean and Transform rev_data

In [26]:
# DF and Rename Columns
rev_data = pd.DataFrame(rev_data)
rev_data = rev_data.rename(columns={"Restaurant": "Store Number"})
rev_data = rev_data.rename(columns={"Overall Score": "Rev Score"})

In [27]:
rev_data = rev_data[["Store Number", "Rev Score"]]

In [28]:
rev_data['Store Number'] = rev_data['Store Number'].astype(str).str[:-2].astype(np.int64)

In [29]:
# Drop bad duplicated data in sa_data
rev_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,Rev Score
0,1440,76.5
2,6496,77.5
4,25099,50.5
6,2881,69.5
8,11697,91.0
...,...,...
996,70650,78.0
998,1116,65.5
1000,24048,80.5
1002,52751,60.0


In [30]:
rev_data.to_excel("data/rev_data_clean.xlsx", index=False)

Clean and Transform Drinks Data

In [6]:
# DF and Rename Columns
drinks_data = pd.DataFrame(drinks_data)
drinks_data = drinks_data.rename(columns={"Restaurant ": "Store Number"})
drinks_data = drinks_data.rename(columns={"Drink % Sales": "Drink Sales"})

In [7]:
drinks_data = drinks_data[["Store Number", "Drink Sales"]]

In [8]:
# Drop bad duplicated data
drinks_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,Drink Sales
0,364,8%
1,1440,8%
2,2031,7%
3,2499,8%
4,2504,9%
...,...,...
640,62703,7%
641,66589,7%
642,67098,11%
643,68704,0%


In [9]:
drinks_data.to_excel("data/drinks_data_clean.xlsx", index=False)

Clean and Transform Rest Data

In [10]:
# DF and Rename Columns
rest_data = pd.DataFrame(rest_data)
rest_data = rest_data.rename(columns={"Restaurant ": "Store Number"})
rest_data = rest_data.rename(columns={"Avg. AUV": "AUV"})
rest_data = rest_data.rename(columns={"Avg. Factor": "Factor"})
rest_data = rest_data.rename(columns={"Avg. Traffic": "Traffic"})
rest_data = rest_data.rename(columns={"Avg. Units Count": "Units"})

In [11]:
rest_data = rest_data[["Store Number", "Average Check", "Food Cost %", "Labor Cost %", "Prod", "AUV", "Factor", "Traffic", "Units"]]

In [12]:
# Drop bad duplicated data
rest_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,Average Check,Food Cost %,Labor Cost %,Prod,AUV,Factor,Traffic,Units
0,1794,11.36,27.9%,20.4%,5.7,8413.167500,7.043090,740.333333,1020.166667
1,11639,12.09,29.9%,19.2%,5.8,7880.895000,7.200319,651.750000,909.333333
2,15186,14.76,28.7%,18.1%,5.7,9748.258333,8.089866,660.250000,1046.333333
3,17473,13.16,29.8%,19.1%,6.0,12652.196667,7.271652,961.500000,1491.916667
4,19922,13.92,28.0%,29.6%,3.7,6224.828333,7.811478,447.250000,706.750000
...,...,...,...,...,...,...,...,...,...
647,54562,9.19,30.4%,28.2%,4.4,2916.326667,7.039648,317.333333,361.000000
648,56810,14.33,28.9%,27.3%,3.4,5533.019167,9.105323,386.000000,511.250000
649,57802,8.10,29.4%,18.7%,4.1,8555.601667,7.398275,1055.666667,651.666667
650,60062,9.10,34.3%,32.9%,4.2,2677.040833,6.756439,294.250000,366.750000


In [13]:
rest_data.to_excel("data/rest_data_clean.xlsx", index=False)

Clean and Transform ro_data

In [14]:
# DF and Rename Columns
ro_data = pd.DataFrame(ro_data)
ro_data = ro_data.rename(columns={"Restaurant Number": "Store Number"})
ro_data = ro_data.rename(columns={"Remote Order Avg Check": "RO Avg Check"})
ro_data = ro_data.rename(columns={"Remote Order Avg Orders": "RO Avg Orders"})
ro_data = ro_data.rename(columns={"Remote Order Avg Sales": "RO Avg Sales"})

In [15]:
ro_data = ro_data[["Store Number", "RO Avg Check", "RO Avg Orders", "RO Avg Sales"]]

In [16]:
# Drop bad duplicated data
ro_data.drop_duplicates(subset=["Store Number"], keep="first")

,Store Number,RO Avg Check,RO Avg Orders,RO Avg Sales
0,261,15.61,55.000000,859
1,276,15.62,85.500000,"1,336"
2,281,14.79,44.333333,656
3,364,13.52,89.666667,"1,212"
4,565,14.73,53.666667,790
...,...,...,...,...
647,70185,11.53,13.666667,158
648,70480,14.04,14.083333,198
649,70650,9.05,58.666667,531
650,71125,14.99,4.250000,64


In [17]:
ro_data.to_excel("data/ro_data_clean.xlsx", index=False)

In [ ]:
# Add closure_data stores to "Closed" Field Consultant parse. If Restaurant Status == "Store Closed" .replace "Closed" in Field Consultant
# This needs to be done after dataset merge, since Field Consultant column is needed
# df.loc[df.Restaurant Status == "Store Closed", "Field Consultant"] = "Closed"

Merge Datasets

In [6]:
# Removed for privacy
# bc_data

In [2]:
# Read in cleaned data sets
win_data_clean = pd.read_excel("data/win_data_clean.xlsx")
ez_data_clean = pd.read_excel("data/ez_data_clean.xlsx")
osat_data_clean = pd.read_excel("data/osat_data_clean.xlsx")
sa_data_clean = pd.read_excel("data/sa_data_clean.xlsx")
hours_data_clean = pd.read_excel("data/hours_data_clean.xlsx")
threepd_data_clean = pd.read_excel("data/3pd_data_clean.xlsx")
bc_data_clean = pd.read_excel("data/BC_list_clean.xlsx")
rev_data_clean = pd.read_excel("data/rev_data_clean.xlsx")
rest_data_clean = pd.read_excel("data/rest_data_clean.xlsx")
drinks_data_clean = pd.read_excel("data/drinks_data_clean.xlsx")
ro_data_clean = pd.read_excel("data/ro_data_clean.xlsx")

In [3]:
#Merge all DFs with Hours_Data Store Number
# dataframes = [hours_data, bc_data, sa_data, osat_data, threepd_data, win_data, ez_data]
# df = reduce(lambda  left,right: pd.merge(left,right,on=['Store Number'], how='inner'), dataframes)
df_bc = hours_data_clean.merge(bc_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(sa_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(win_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(osat_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(threepd_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(ez_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(rev_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(rest_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(drinks_data_clean, on="Store Number", how="left")
df_bc = df_bc.merge(ro_data_clean, on="Store Number", how="left")

In [4]:
df_bc.columns

Index(['Store Number', 'Total Hours', 'BD Contract_x', 'Primary Owner Name',
       'Restaurant', 'Field Consultant', 'SA1 Overdue', 'SA1 Complete',
       'SA2 Complete', 'Window Complete', 'Overall Satisfaction', 'Quality',
       'Speed of Service', 'Experience with Staff', 'Cleanliness', 'NPS',
       'Alerts Closed 24 Hours', 'Surveys Per Restaurant', 'Grubhub Status',
       'Subway Delivery Status', 'Doordash Status', 'Uber Status',
       'BD Contract_y', 'ezCater', 'Delivery', 'ezDispatch', 'Owner Name',
       'Rev Score', 'Average Check', 'Food Cost %', 'Labor Cost %', 'Prod',
       'AUV', 'Factor', 'Traffic', 'Units', 'Drink Sales', 'RO Avg Check',
       'RO Avg Orders', 'RO Avg Sales'],
      dtype='object')

In [7]:
# Removed for privacy
# df_bc.drop_duplicates(subset=["Store Number"], keep="first")

Clean master df

In [8]:
# Removed for privacy
# df_bc = pd.DataFrame(df_bc)
# df_bc["Field Consultant"].fillna("Unknown", inplace=True)
# df_bc["Field Consultant"].replace("###", inplace=True)
# df_bc.drop_duplicates(subset=["Store Number"], keep="first", inplace=True)
# test_unique = df_bc["Field Consultant"].unique()

In [8]:
# Fix breaks from old df_bc to new df_bc
df_bc = df_bc.rename(columns={"BD Contract": "BD Contract_x"})
df_bc = df_bc.rename(columns={"BD Contract_y": "BD Contract "})
df_bc = df_bc.rename(columns={"ezDispatch": "ezDisptach"})

In [9]:
df_bc.to_excel("data/new_df_bc.xlsx", index=False)

Send Field Consultant Output

In [41]:
df = pd.read_excel("data/df.xlsx")

In [9]:
# df_bc_unique = df_bc["Field Consultant"].unique()
# df_bc["Field Consultant"].fillna("Unknown", inplace=True)
# df_bc

In [45]:
for i in test_unique:
    a = df_bc[df_bc["Field Consultant"].str.contains(i)]
    a.to_excel("data/output/test/bc_dash/" + i + ".xlsx", index=False)

Miscellaneous

In [24]:
# Write DF to computer
# osat_data.to_excel("osat_data.xlsx")

In [32]:
# bc_data.to_excel("data/new_BC_list.xlsx", index=False)

In [121]:
# bc_data.drop_duplicates(subset=["Store Number"], keep="first")

In [132]:
# Create Duplicates DF for finding Repeats and NaNs or printout whole bc_data list (if no Repeats or NaNs)
# bc_dup = pd.concat(g for _, g in bc_data.groupby("Field Consultant") if len(g) > 1)
# bc_dup.to_excel("data/bc_duplicates.xlsx", index=False)

In [9]:
# Saving without Index
# hours_data.to_excel("hours_data.xlsx", index=False)

In [90]:
# df.to_excel("data/test_df.xlsx", index=False)

In [36]:
# df.to_excel("df.xlsx")

In [104]:
# df.to_excel("df.xlsx", index=False)

In [ ]:
# Drop one column in place
# ez_data.drop(["METRO"], axis=1, inplace=True)